In [155]:
import pandas as pd
import niquests
import urllib.parse
from IPython.display import display

In [205]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [137]:
def validate_urls(src_url, tgt_url):
    print(f"src_url:{src_url} \ntgt_url:{tgt_url}")
    assert src_url.netloc != tgt_url.netloc
    assert src_url.path == tgt_url.path
    assert src_url.params == tgt_url.params
    assert src_url.query == tgt_url.query
    assert src_url.fragment == tgt_url.fragment

In [159]:
def fetch_data(src_url, tgt_url):
    src_json = niquests.get(
        src_url.geturl(), 
        headers={"Accept": "application/json", "Content-Type": "application/json"},
        timeout=5,
    )
    src_df = pd.DataFrame(src_json.json()['result'])
    print("***** Source Data Sample:")
    display(src_df.sample())
    
    tgt_json = niquests.get(
        tgt_url.geturl(), 
        headers={"Accept": "application/json", "Content-Type": "application/json"},
        timeout=5,
    )
    tgt_df = pd.DataFrame(tgt_json.json()['result'])
    print("***** Target Data Sample:")
    display(tgt_df.sample())
    
    print("\n")
    return (src_df, tgt_df)

In [197]:
def clean_df(df, excluded_cols):
    colFilter = df.filter(excluded_cols)
    df = df.drop(colFilter, axis=1)
    if 'fid' in df.columns:
        df['fid'] = df['fid'].astype(dtype='int32')
    return df

In [193]:
def compare_data(src_df, tgt_df, excluded_cols):
    src_df = clean_df(src_df, excluded_cols)
    tgt_df = clean_df(tgt_df, excluded_cols)
    cmp_df = src_df.compare(tgt_df, result_names=('src', 'tgt'), align_axis=1)
    display(tgt_df.sample())
    return cmp_df

In [194]:
def validate_fetch_compare(src_url, tgt_url):
    validate_urls(src_url, tgt_url)
    (src_df, tgt_df) = fetch_data(src_url, tgt_url)
    return (src_df, tgt_df, compare_data(src_df, tgt_df, ['bal_update_ts']))

# Channels

In [117]:
channel_id = input()

 base


## Stats

In [160]:
src_url = urllib.parse.urlparse(f"https://graph.cast.k3l.io/channels/rankings/{channel_id}/stats?rank_timeframe=60d")

In [161]:
tgt_url = urllib.parse.urlparse(f"https://graph.cast10.k3l.io/channels/rankings/{channel_id}/stats?rank_timeframe=60d")

In [162]:
validate_fetch_compare(src_url, tgt_url)

src_url:ParseResult(scheme='https', netloc='graph.cast.k3l.io', path='/channels/rankings/base/stats', params='', query='rank_timeframe=60d', fragment='') 
tgt_url:ParseResult(scheme='https', netloc='graph.cast10.k3l.io', path='/channels/rankings/base/stats', params='', query='rank_timeframe=60d', fragment='')
***** Source Data Sample:


,channel_id,num_followers,num_fids_ranked,min_score,max_score,num_members,num_holders,min_balance,max_balance,token_num_holders,token_min_balance,token_max_balance
0,base,440354,3609,0.00,0.18,714,3871,0.00,"119,049.81",None,None,None


***** Target Data Sample:


,channel_id,num_followers,num_fids_ranked,min_score,max_score,num_members,num_holders,min_balance,max_balance,token_num_holders,token_min_balance,token_max_balance
0,base,419421,3609,0.00,0.18,716,3871,0.00,"119,075.81",None,None,None


num_followers  num_members          max_balance
0 src         440354          714           119,049.81
  tgt         419421          716           119,075.81

## Rankings

In [157]:
src_url = urllib.parse.urlparse(f"https://graph.cast.k3l.io/channels/rankings/{channel_id}"
                                "?rank_timeframe=60d&offset=0&limit=1000&lite=true")
tgt_url = urllib.parse.urlparse(f"https://graph.cast10.k3l.io/channels/rankings/{channel_id}"
                                "?rank_timeframe=60d&offset=0&limit=1000&lite=true")

In [158]:
validate_fetch_compare(src_url, tgt_url)

src_url:ParseResult(scheme='https', netloc='graph.cast.k3l.io', path='/channels/rankings/base', params='', query='rank_timeframe=60d&offset=0&limit=1000&lite=true', fragment='') 
tgt_url:ParseResult(scheme='https', netloc='graph.cast10.k3l.io', path='/channels/rankings/base', params='', query='rank_timeframe=60d&offset=0&limit=1000&lite=true', fragment='')
***** Source Data Sample:



,fid,rank
281,238814,282


***** Target Data Sample:



,fid,rank
942,262876,943


,


## Holders

In [188]:
src_url = urllib.parse.urlparse(f"https://graph.cast.k3l.io/channels/holders/{channel_id}"
                                "?rank_timeframe=60d&orderby=total&offset=0&limit=100")
tgt_url = urllib.parse.urlparse(f"https://graph.cast10.k3l.io/channels/holders/{channel_id}"
                                "?rank_timeframe=60d&orderby=total&offset=0&limit=100")

In [210]:
(src_df, tgt_df, cmp_df) = validate_fetch_compare(src_url, tgt_url)

src_url:ParseResult(scheme='https', netloc='graph.cast.k3l.io', path='/channels/holders/base', params='', query='rank_timeframe=60d&orderby=total&offset=0&limit=100', fragment='') 
tgt_url:ParseResult(scheme='https', netloc='graph.cast10.k3l.io', path='/channels/holders/base', params='', query='rank_timeframe=60d&orderby=total&offset=0&limit=100', fragment='')
***** Source Data Sample:


,fid,fname,username,pfp,bio,channel_id,rank,score,global_rank,addresses,balance,token_balance,daily_earnings,token_daily_earnings,is_weekly_earnings_available,latest_earnings,token_latest_earnings,weekly_earnings,token_weekly_earnings,bal_update_ts,is_points_launched,is_tokens_launched,memberat,order_rank
85,277293,silvermen,silvermen,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/e095a6da-c089-45e0-df31-99cc28337500/rectcrop3,NFT Investor | Crypto Trader | Builder on Base\nBringing the world onchain\n0xNot of /base,base,267.00,0.00,36491,"[0x0dbd9ceccd76eb45d69895b1fb5822abcd572a70, 0x719aa940beddc87ea07d95da56b4f85cd5fa9855]","1,078.01",None,43.00,0,True,None,None,59,0,2025-03-12T17:00:15.955544,True,False,"1,736,425,723.00",86


***** Target Data Sample:


,fid,fname,username,pfp,bio,channel_id,rank,score,global_rank,addresses,balance,token_balance,daily_earnings,token_daily_earnings,is_weekly_earnings_available,latest_earnings,token_latest_earnings,weekly_earnings,token_weekly_earnings,bal_update_ts,is_points_launched,is_tokens_launched,memberat,order_rank
8,306404,harrytrananh,harrytrananh,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/7a4256f0-dbf4-45e4-3c98-d169b2f63f00/original,"Proudly data scientist, Trust Wallet Program Coordinator for SEA & Aus | dreaming to become an NFT artist | Based Ambassador building /based-vietnamese",base,25.00,0.00,789,"[0x21d578fc404c331c3ff910c4f7ffb4a8ee8c18c1, 0xecaa5f28aa25cc39a848836b32744673a555b9ba]","5,814.83",None,350.00,0,True,None,None,637,0,2025-03-12T17:00:12.127364,True,False,"1,725,241,781.00",9


,fid,fname,username,pfp,bio,channel_id,rank,score,global_rank,addresses,balance,token_balance,daily_earnings,token_daily_earnings,is_weekly_earnings_available,latest_earnings,token_latest_earnings,weekly_earnings,token_weekly_earnings,is_points_launched,is_tokens_launched,memberat,order_rank
62,561092,loredangoo,loredangoo,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/e0508a54-1381-4f77-e8a3-40139efc2b00/rectcrop3,"Illustrator & Animator\n| Kawaii but not really\n| Bureau of Internet Culture\n| Part of Anifam, Artgumi & bpdotfun\n| Find art, merch & more at loredangoo.carrd.co",base,44.00,0.00,3902,"[0x7c1bc3de66dac578593ce918dfcd6d7508df0688fea4dc4adf456bc223a33dec, 0xce041df72132178afed6fdecb0331200a1b439c3, 0xfc92b5faa50350d8dd1aef8573c50faa51eb9260]","1,469.21",None,0.00,0,True,None,None,0,0,True,False,NaN,63


In [211]:
cmp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 0 to 99
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   (fid, src)              21 non-null     float64
 1   (fid, tgt)              21 non-null     float64
 2   (fname, src)            31 non-null     object 
 3   (fname, tgt)            31 non-null     object 
 4   (username, src)         21 non-null     object 
 5   (username, tgt)         21 non-null     object 
 6   (pfp, src)              23 non-null     object 
 7   (pfp, tgt)              23 non-null     object 
 8   (bio, src)              21 non-null     object 
 9   (bio, tgt)              21 non-null     object 
 10  (rank, src)             20 non-null     float64
 11  (rank, tgt)             20 non-null     float64
 12  (score, src)            20 non-null     float64
 13  (score, tgt)            20 non-null     float64
 14  (global_rank, src)      22 non-null     float64
 

In [212]:
cmp_df

fid                                   fname                              username                                                                                                                 pfp                                                                                                                                                                                                                                                   bio                                                                                                                                                                      rank                                     score                               global_rank                                                                                                                                                                                                                                                                                                              addresses                                                                                                                                                                                                                                                                                                                balance                            daily_earnings                           weekly_earnings                                  memberat                     
                    src                  tgt                src                tgt                src                tgt                                                                                              src                                                                                              tgt                                                                                                                                                  src                                                                                                                                                  tgt                  src                  tgt                  src                  tgt                  src                  tgt                                                                                                                                                                                                                                                                                               src                                                                                                                                                                                                                                                                                               tgt                  src                  tgt                  src                  tgt                  src                  tgt                  src                  tgt
0                   NaN                  NaN                NaN                NaN                NaN                NaN                                                                                              NaN                                                                                              NaN                                                                                                                                                  NaN                                                                                                                                                  NaN                  NaN                  NaN                  NaN                  NaN                  NaN                  NaN                                                                                                                                                                                                                                                                                               NaN                  

In [208]:
src_df.loc[[8, 13, 16]]

,fid,fname,username,pfp,bio,channel_id,rank,score,global_rank,addresses,balance,token_balance,daily_earnings,token_daily_earnings,is_weekly_earnings_available,latest_earnings,token_latest_earnings,weekly_earnings,token_weekly_earnings,bal_update_ts,is_points_launched,is_tokens_launched,memberat,order_rank
8,306404,harrytrananh,harrytrananh,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/7a4256f0-dbf4-45e4-3c98-d169b2f63f00/original,"Proudly data scientist, Trust Wallet Program Coordinator for SEA & Aus | dreaming to become an NFT artist | Based Ambassador building /based-vietnamese",base,25.00,0.00,789,"[0x21d578fc404c331c3ff910c4f7ffb4a8ee8c18c1, 0x90c0623eaee787d7ec9f8997073ebb5d0cb6a0e23fcfd0b68845cdffe1646003, 0xecaa5f28aa25cc39a848836b32744673a555b9ba]","5,784.83",None,342.00,0,True,None,None,628,0,2025-03-12T17:00:15.955544,True,False,"1,725,241,781.00",9
13,379435,with-dani69,with-dani69,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/f44a766c-1fd6-46b5-ca2d-7b3f8254a000/rectcrop3,Welcome to my Surreal World |Photographer | Veterinarian | \nArtist Profile and NFTs \n\n\nlinktr.ee/adnan_dani,base,20.00,0.00,1238,"[0x0e51e0981b2a8a8be594ee0882a654ca17b48ab4, 0x24255d06cfb9dbae774f9fb8697f7850f1d8a5f1, 0xca328c7c4bc086629e4c9881b27d92f56cd5f70f, 0xdad60277896e3577f2e4de935bb033a1010e69e6]","4,811.07",None,158.00,0,True,None,None,314,0,2025-03-12T17:00:15.955544,True,False,"1,736,142,475.00",14
16,348569,ali-sharafi,ali-sharafi,https://imagedelivery.net/BXluQx4ige9GuW0Ia56BHw/ed9e16a4-73a8-407c-c796-92bca5719400/rectcrop3,https://www.instagram.com/nabu.base.eth,base,59.00,0.00,1856,"[0x810b71bcd1cdbab21a3cacc95c900390b31181bd, 0xa5ef58e2d291c2d0481c3087535a211fd81a8e2e, 0xacbf5b79db3f441b3e1f0784f419ade0027ed899, 0xb44b38258c4b97e6aa4aa1a9e223d98f5f8b9feedf5fe1013a38a1b2b62efa99]","3,464.92",None,53.00,0,True,None,None,160,0,2025-03-12T17:00:15.955544,True,False,"1,729,082,011.00",17
